<a href="https://colab.research.google.com/github/tukamilano/pangram_kenlm/blob/main/pangram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fugashi unidic-lite

In [5]:
import requests

# Hugging Face上のファイルURL
url0 = "https://huggingface.co/milano0017/hiragana_KenLM/resolve/main/model.bin"
url1 = "https://huggingface.co/milano0017/hiragana_KenLM/resolve/main/reversed_model.bin"

# 保存先のファイル名
destination_file_name0 = "model.bin"
destination_file_name1 = "reversed_model.bin"

# ファイルをダウンロード
response = requests.get(url0, stream=True)
if response.status_code == 200:
    with open(destination_file_name0, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"'{destination_file_name0}' をダウンロードしました。")
else:
    print(f"エラー: ダウンロードに失敗しました (status code: {response.status_code})")

response = requests.get(url1, stream=True)
if response.status_code == 200:
    with open(destination_file_name1, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"'{destination_file_name1}' をダウンロードしました。")
else:
    print(f"エラー: ダウンロードに失敗しました (status code: {response.status_code})")


'model.bin' をダウンロードしました。
'reversed_model.bin' をダウンロードしました。


In [6]:
!sudo apt-get install -y libboost-all-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libboost-atomic-dev libboost-atomic1.74-dev
  libboost-atomic1.74.0 libboost-chrono-dev libboost-chrono1.74-dev
  libboost-chrono1.74.0 libboost-container-dev libboost-container1.74-dev
  libboost-container1.74.0 libboost-context-dev libboost-context1.74-dev
  libboost-context1.74.0 libboost-coroutine-dev libboost-coroutine1.74-dev
  libboost-coroutine1.74.0 libboost-date-time-dev libboost-date-time1.74-dev
  libboost-date-time1.74.0 libboost-exception-dev libboost-exception1.74-dev
  libboost-fiber-dev libboost-fiber1.74-dev libboost-fiber1.74.0
  libboost-filesystem-dev libboost-filesystem1.74-dev
  libboost-filesystem1.74.0 libboost-graph-dev libboost-graph-parallel-dev
  libboost-graph-parallel1.74-dev libboost-graph-parallel1.74.0
  libboost-graph1.74-dev libboost-graph1.74.0 libboost-iostreams-dev
  libboost-iost

In [7]:
import os

# Clone KenLM if not already cloned
if not os.path.exists("kenlm"):
    !git clone https://github.com/kpu/kenlm.git
    print("KenLM repository cloned.")
else:
    print("KenLM repository already exists.")

# Build KenLM
# Change to kenlm directory
%cd kenlm

# Create build directory and navigate into it
!mkdir -p build
%cd build

# Configure and build KenLM
!cmake ..
!make -j $(nproc)

# Navigate back to the original content directory
%cd /content

print("KenLM build process complete. You can now try to run the lmplz command.")

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 14185 (delta 117), reused 81 (delta 81), pack-reused 14009 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.
KenLM repository cloned.
/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeLis

上のコードでエラーが出ても続けて大丈夫

In [8]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     | 553.6 kB 3.3 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp312-cp312-linux_x86_64.whl size=3188042 sha256=5765d7251a6b3332ee20e742cb18126d6397e5b21e8a5b6ab9fc6ae1dd9570c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-decc3np8/wheels/92/c8/12/56d187154e078f0eaa74d059017fc1afe1c4d91fbce02ce8d9
Successfully built kenlm


In [9]:
import kenlm

model = kenlm.Model("model.bin")

In [10]:
import fugashi

# ==== 初期化 ====
print("形態素解析器を初期化中...")
tagger = fugashi.Tagger()

形態素解析器を初期化中...


In [11]:
import random
import math
from itertools import combinations

# ===============
# ひらがなセット
# ===============
jp = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")

# ===============================
# scoreを計算する関数
# ===============================
def calculate_score(sentence):
    """
    KenLM を使って
    与えられた文の score（自然さ）を算出する.
    """
    sentence = ''.join(sentence)
    processed_sentence_list = [word.surface for word in tagger(sentence)]
    reversed_processed_sentence_list = processed_sentence_list[::-1]
    processed_sentence = ' '.join(processed_sentence_list)
    reversed_processed_sentence = ' '.join(reversed_processed_sentence_list)
    score = -model.score(processed_sentence) -model.score(reversed_processed_sentence)
    return score

In [18]:
import json

def improve_by_4_opt(sentence, step=1000000):
    v = calculate_score(sentence)
    calculate_num = 0
    improved = True

    while improved:
        improved = False
        indices = list(range(len(sentence)+1))
        base = list(combinations(indices, 4))
        random.shuffle(base)  # 組の順序をランダム化

        for four_var in base:
            a, b, c, d = four_var
            new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
            new_v = calculate_score(new_sentence)
            calculate_num += 1

            if new_v < v:
                v = new_v
                sentence = new_sentence
                improved = True
                print(f"Improved! step={calculate_num}, new_score={v:.3f}")
                break

            if calculate_num >= step:
                print(f"Reached step limit ({step}). Stop.")
                return sentence

    print("No more improvement.")
    return sentence

def improve_by_4_opt_sa(sentence, step=2_000_000, T0=2.0, Tmin=0.1, alpha=0.999999):

    save_path = "best_sentences.jsonl"
    with open(save_path, "w") as f:
        pass

    v = calculate_score(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        indices = sorted(random.sample(range(len(sentence)+1), 4))
        a, b, c, d = indices
        new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_score(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_score={v:.3f}, best sentence = {''.join(best_sentence)}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            record = {
                "step": step_num,
                "T": T,
                "score": float(best_v),
                "sentence": "".join(best_sentence)
            }

            with open(save_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(record, ensure_ascii=False) + "\n")
            print(f"Step {step_num}: T={T:.3f}, score={v:.3f}, sentence = {''.join(sentence)}")

    print(f"Finished all {step} steps.")
    print(f"Best score: {best_v:.3f}")
    return best_sentence

def improve_by_4_opt_sa_select_by_morph(sentence, step=1_000_000, T0=1000.0, Tmin=0.1, alpha=0.99999, morph_ratio=0.5):
    v = calculate_score(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        if random.random() < morph_ratio:
          processed_sentence_list = [word.surface for word in tagger(''.join(sentence))]
          indices = sorted(random.sample(range(len(processed_sentence_list)+1), 4))
          a, b, c, d = indices
          new_processed_sentence_list = processed_sentence_list[:a] + processed_sentence_list[c:d] + processed_sentence_list[b:c] + processed_sentence_list[a:b] + processed_sentence_list[d:]
          new_sentence = ''.join(new_processed_sentence_list)
        else:
          indices = sorted(random.sample(range(len(sentence)+1), 4))
          a, b, c, d = indices
          new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_score(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_score={v:.3f}, best sentence = {best_sentence}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            print(f"Step {step_num}: T={T:.3f}, score={v:.3f}, sentence = {sentence}")

    print(f"Finished all {step} steps.")
    print(f"Best score: {best_v:.3f}")
    return best_sentence

# ========================
# 実行部分
# ========================
def main():
    sentence = jp[:]  # コピーしておく
    random.shuffle(sentence)
    print("Initial sentence:", "".join(sentence))
    improved_sentence = improve_by_4_opt_sa(sentence)
    print("Final sentence:", "".join(improved_sentence))

if __name__ == "__main__":
    main()


Initial sentence: くかよもへすにそまむひけちわめつはゆぬんせてきたあしるねほれのさうえこみろおらなやいをりふと
Step 0: T=2.000, score=235.801, sentence = くかよさうえこみろめつはゆぬんせてきたあしるねほれのもへすにそまむひけちわおらなやいをりふと
Step 1: Improved! new_score=233.451, best sentence = くかよれのもへすにそまはゆぬんせてきたあしるねほさうえこみろめつむひけちわおらなやいをりふと
Step 3: Improved! new_score=226.395, best sentence = くかよれのもへすにそまたあしきはゆぬんせてるねほさうえこみろめつむひけちわおらなやいをりふと
Step 7: Improved! new_score=223.113, best sentence = ねほさうえこみろめつむひけちわおらなやいをあしきはゆぬんせてるくかよれのもへすにそまたりふと
Step 11: Improved! new_score=220.958, best sentence = ねほさもへしきはゆぬんせてるおらなやいをあすにそまわくかよれのうえこみろめつむひけちたりふと
Step 13: Improved! new_score=220.174, best sentence = すにそまわくかよれのうえこみやいをあねなろめつむひけらほさもへしきはゆぬんせてるおちたりふと
Step 16: Improved! new_score=218.116, best sentence = すつむひけらほさもへねなろめにそまわくかよれのうえこみやいをあしきはゆぬんせてるおちたりふと
Step 17: Improved! new_score=217.623, best sentence = すつむひけらほさもへねなろめにそまわくかよれのうえこみやいをあしきせてるおちたりふゆぬんはと
Step 27: Improved! new_score=216.332, best sentence = すつむひけらほさもへぬまわくかよたりふゆねれのうえこみやいをあしきせてるにそちなろめおんはと
Step 29: Improved! new_score=211.940, be

In [19]:
text = "おまえはもちぬしのそうさくてきかんせいにほらあなをむねへふところよひけるゆすりみつれたわやめ"
processed_sentence = ' '.join(word.surface for word in tagger(text))
print(processed_sentence)

おまえ は もちぬし の そうさく てき かんせい に ほらあな を むね へ ふところ よ ひける ゆすりみつれ たわやめ
